

DataBase Loaders
SQLDatabaseLoader
Definition: A loader that connects to SQL databases (Postgres, MySQL, SQLite, etc.) and extracts table rows into documents for use with LLMs.

Usage: Use this when you want to bring raw SQL database content into LangChain for retrieval or fine-tuning.

Parameters:

sql_database: SQLAlchemy SQLDatabase object

query: SQL query (string) to fetch data

###  First Create SQLite DB

In [2]:
import sqlite3

conn = sqlite3.connect("11_sqlite_db.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    income REAL,
    spending_score INTEGER
);
""")


# Insert dummy data
dummy_customers = [
    ("Ali", 25, 30000, 45),
    ("Sara", 32, 60000, 80),
    ("Ahmed", 40, 120000, 20),
    ("Ayesha", 29, 45000, 60),
    ("Bilal", 50, 90000, 30)
]

cursor.executemany("INSERT INTO customers (name, age, income, spending_score) VALUES (?, ?, ?, ?);", dummy_customers)

conn.commit()
conn.close()

### Now Load With SQLDatabaseLoader

In [3]:
from langchain_community.document_loaders import SQLDatabaseLoader
from langchain_community.utilities import SQLDatabase

# Step 1: Connect to your existing DB 
db = SQLDatabase.from_uri("sqlite:///11_sqlite_db.db")

# Step 2: Load data with SQLDatabaseLoader
loader = SQLDatabaseLoader(db=db,  query="SELECT * FROM customers;")

docs = loader.load()

# Step 3: Print results 
for doc in docs:
    print(doc.page_content)  

c:\Users\Lenovo\OneDrive\Desktop\LangChain_Model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


id: 1
name: Ali
age: 25
income: 30000.0
spending_score: 45
id: 2
name: Sara
age: 32
income: 60000.0
spending_score: 80
id: 3
name: Ahmed
age: 40
income: 120000.0
spending_score: 20
id: 4
name: Ayesha
age: 29
income: 45000.0
spending_score: 60
id: 5
name: Bilal
age: 50
income: 90000.0
spending_score: 30


### Mongo Db loader

## First Create Mongo DB

In [4]:

%pip install pymongo

from pymongo import MongoClient

#  Step 1: Connect to MongoDB 
client = MongoClient("mongodb://localhost:27017/")

#  Step 2: Create database and collection 
db = client["testdb"]
collection = db["customers"]

#  Step 3: Insert dummy data 
dummy_customers = [
    {"name": "Ali", "age": 25, "income": 30000, "status": "active"},
    {"name": "Sara", "age": 32, "income": 60000, "status": "active"},
    {"name": "Ahmed", "age": 40, "income": 120000, "status": "inactive"},
    {"name": "Ayesha", "age": 29, "income": 45000, "status": "active"},
    {"name": "Bilal", "age": 50, "income": 90000, "status": "inactive"}
]

collection.insert_many(dummy_customers)

print("MongoDB test database created with dummy customers.")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pymongo-4.16.0-cp313-cp313-win_amd64.whl.metadata (10.0 kB)
  Using cached dnspython-2.8.0-py3-none-any.whl.metadata (5.7 kB)
Using cached pymongo-4.16.0-cp313-cp313-win_amd64.whl (959 kB)
Using cached dnspython-2.8.0-py3-none-any.whl (331 kB)

   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   -----------------

In [8]:
docs = await loader.aload()

for doc in docs:
    print("Content:", doc.page_content)


Content: Ali 25 30000 active
Content: Sara 32 60000 active
Content: Ahmed 40 120000 inactive
Content: Ayesha 29 45000 active
Content: Bilal 50 90000 inactive
